# **Step 1: Install Dependencies**
Before running the code, ensure you have the necessary packages installed.

## Install Ollama
Ollama is required for embeddings and chat-based interactions.

```bash
# Install Ollama
curl -fsSL https://ollama.com/install.sh | sh

# Pull the DeepSeek-R1 model
ollama pull deepseek-r1



---

### **2️⃣ Load the PDF and Create Embeddings**

# **Step 2: Load and Process the PDF**
We will:
1. Load a PDF using `PyMuPDFLoader`.
2. Split it into smaller text chunks for efficient retrieval.
3. Generate embeddings using `OllamaEmbeddings` with DeepSeek-R1.
4. Store these embeddings in a **ChromaDB** vector database.


In [ ]:
!pip install gradio langchain langchain-community chromadb pymupdf

In [10]:
import re
from concurrent.futures import ThreadPoolExecutor
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from chromadb.config import Settings
from chromadb import Client

# Load the document
loader = PyMuPDFLoader("document-20-24.pdf")
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# Initialize embeddings model
embedding_function = OllamaEmbeddings(model="deepseek-r1")

# Generate embeddings in parallel
def generate_embedding(chunk):
    return embedding_function.embed_query(chunk.page_content)

with ThreadPoolExecutor() as executor:
    embeddings = list(executor.map(generate_embedding, chunks))

# Initialize Chroma client
client = Client(Settings())

# Delete and create new collection
try:
    client.delete_collection(name="foundations_of_llms")
except ValueError as e:
    print(f"Error deleting collection: {e}")

collection = client.create_collection(name="foundations_of_llms")

# Add documents and embeddings to Chroma
for idx, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk.page_content], 
        metadatas=[{'id': idx}], 
        embeddings=[embeddings[idx]], 
        ids=[str(idx)]
    )

# Initialize retriever
retriever = Chroma(collection_name="foundations_of_llms", client=client, embedding_function=embedding_function).as_retriever()


# **Step 3: Context Retrieval and Chat**
Now, we will:
1. **Retrieve** relevant sections from ChromaDB based on user queries.
2. **Generate responses** using `Ollama` with DeepSeek-R1.


In [11]:
# Initialize LLM for answering questions
llm = Ollama(model="deepseek-r1")

def retrieve_context(question):
    """Retrieve relevant context from stored embeddings."""
    results = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in results])
    return context

def query_deepseek(question, context):
    """Use DeepSeek-R1 to generate an answer."""
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    
    # Generate response
    response = llm.invoke(formatted_prompt)

    # Clean response
    final_answer = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL).strip()
    return final_answer

def ask_question(question):
    """Retrieve context and answer the question."""
    context = retrieve_context(question)
    answer = query_deepseek(question, context)
    return answer


/tmp/ipykernel_11503/3410436889.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1")


# **Step 4: Deploy the Chatbot UI**
Finally, we will use **Gradio** to create a simple web interface where users can input their questions and receive answers based on the processed PDF.


In [14]:
# Set up the Gradio interface
interface = gr.Interface(
    fn=ask_question,
    inputs="text",
    outputs="text",
    title="Volvo Chatbot: Troubleshooting",
    description="Ask any question about the Volvo manual book. Powered by DeepSeek-R1."
)

# Launch the interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


# Final Notes

    📄 This notebook will process the PDF once and store embeddings in ChromaDB, so you don’t need to reprocess it every time.
    ⚡ The chatbot retrieves only relevant sections before generating an answer using DeepSeek-R1.
    🚀 Gradio provides an easy-to-use UI for interacting with the chatbot.

Try running each cell one by one in a Jupyter Notebook, and let me know if you need any modifications! 🚀